### Multi-Class Classification
Now that we've explored a binary classification problem with linear and non-linear architectures, we now want to shift to a multi-class problem where there are more than two options that the model needs to be able to classify.

The multi-class data will be artificial data from the scikit-learn `make_blobs()` function. The general flow is as follows:
1. Make the artificial data and convert to tensors
2. Visualize the data
3. Define the model architecture
4. Train the model
5. Adjust hyperparameters as necessary

In [ ]:
# First, lets explore the make_blobs() function. According to the documentation,
# make_blobs is designed for creating artificial multiclass data by creating isotropic, Gaussian clusters
# of points. The data is quite literally "blobs" of points around a "center" in R^n space. The classes could be based 
# on the number of centers in a feature set.

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

n_blobs = 50
X_blob, y_blob = make_blobs([n_blobs, n_blobs], n_features=3, centers=None, random_state=42)

fig = plt.figure()
plt.title('Two blobs with 3 features (x, y, z)')
ax = fig.add_subplot(projection='3d')
ax.scatter(X_blob[:,0], X_blob[:,1], X_blob[:, 2], c=y_blob)

In [ ]:
# Now lets standardize this a little for the actual model. Will define the const values that will be used when creating
# the architecture (allow things to be updated once). This could probably eventually be refactored into a dataclass.

NUM_CLASSES = 4 # This is self explanatory, the number of blobs per training data instance
CLUSTER_POINTS = 100 # This is the number of points that are in each blob
NUM_FEATURES = 2 # This refers to the dimension of the data. In this case, the dimension of the the points in the blobs (the above example is 3D)
CLUSTER_STD_DEV = 1.0 # This changes the spread in each blob (makes classification more difficult!)
RANDOM_SEED = 42

In [ ]:
# Now lets create our training data and move to tensors